# Lab 12

- **DP02_0037E** -> Number of women 15-50 years old who had a birth in the past 12 months
- **DP02_0037PE**  -> Percent of women 15-50 years old who had a birth in the past 12 months. 
- **DP02_0040E** -> Number of women 15 to 50 years old who had a birth in the past 12 months, Per 1,000 women 15 to 50 years old 

In [1]:
import pandas as pd
import json
import requests
import pprint

import plotly.express as px

# 1. Build the API Request URL

### 1. How to Start your Data Request  
- Requests always begin with: https://api.census.gov/data  

In [2]:
base_url = "https://api.census.gov/data"

### 2. Add the Dataset Name


In [3]:
dataset_name = "/2020/acs/acs5/profile"

### 3. Start your Variable Request


In [4]:
get_start = "?get="

### 4. Add your Variables  
- **DP02_0066PE**: Percent of Population 25 years and over with Graduate or professional degree


In [5]:
get_variables = "NAME,DP02_0037E,DP02_0037PE,DP02_0040E"

###   5. Add your Geography  


In [6]:
geography = "&for=state:*"

###   6. Put it all Together 

In [7]:
request_url = base_url + dataset_name + get_start + get_variables + geography
print("request_url = ", request_url)

request_url =  https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,DP02_0037E,DP02_0037PE,DP02_0040E&for=state:*


# 2. Use *requests* library to make the API call

In [8]:
# Make API Call
r = requests.get(request_url)

api_results = r.json()

In [9]:
#print(api_results)

In [10]:
# pprint makes it possible to see the structure of the returned data -- but it can be very, very long!
#pprint.pprint(api_results)

In [11]:
type(api_results)

list

# 3. Get the data into a Dataframe  
- These Census Data results are in a list and have a specific form:  
  - The first element is a list of column names  
  - The remaining list elements are data  
  
  

In [12]:
df = pd.DataFrame(api_results)

print(df.shape)
df.head()

(53, 5)


,0,1,2,3,4
0,NAME,DP02_0037E,DP02_0037PE,DP02_0040E,state
1,Arkansas,38542,38542,56,05
2,Washington,96936,96936,55,53
3,Kansas,38807,38807,59,20
4,Oklahoma,54037,54037,59,40


# 4. Get the first Row into columns and then get rid of it  

In [13]:
df.columns = df.iloc[0]

df =df.iloc[1:]

print(df.shape)
df.head()

(52, 5)


,NAME,DP02_0037E,DP02_0037PE,DP02_0040E,state
1,Arkansas,38542,38542,56,05
2,Washington,96936,96936,55,53
3,Kansas,38807,38807,59,20
4,Oklahoma,54037,54037,59,40
5,Wisconsin,68361,68361,53,55


In [14]:
df.dtypes

0
NAME           object
DP02_0037E     object
DP02_0037PE    object
DP02_0040E     object
state          object
dtype: object

# 5. Cleaning Data

In [15]:
names = ['District of Columbia', 'Puerto Rico']
df.query('NAME != @names', inplace=True)

print(df.shape)
df.head()

(50, 5)


,NAME,DP02_0037E,DP02_0037PE,DP02_0040E,state
1,Arkansas,38542,38542,56,05
2,Washington,96936,96936,55,53
3,Kansas,38807,38807,59,20
4,Oklahoma,54037,54037,59,40
5,Wisconsin,68361,68361,53,55


In [16]:
cols_to_rename = {
                   'NAME':'State_Name', 
                   'state':'FIPS_State'
                 }
df.rename(columns = cols_to_rename, inplace=True)

print(df.shape)
df.head()

(50, 5)


,State_Name,DP02_0037E,DP02_0037PE,DP02_0040E,FIPS_State
1,Arkansas,38542,38542,56,05
2,Washington,96936,96936,55,53
3,Kansas,38807,38807,59,20
4,Oklahoma,54037,54037,59,40
5,Wisconsin,68361,68361,53,55


In [17]:
df["DP02_0037E"] = pd.to_numeric(df['DP02_0037E'])
df["DP02_0037PE"] = pd.to_numeric(df['DP02_0037PE'])
df["DP02_0040E"] = pd.to_numeric(df['DP02_0040E'])

In [18]:
df["DP02_0040E"] = df['DP02_0040E'] / 100

In [19]:
df.dtypes

0
State_Name      object
DP02_0037E       int64
DP02_0037PE      int64
DP02_0040E     float64
FIPS_State      object
dtype: object

In [20]:
df.head()

,State_Name,DP02_0037E,DP02_0037PE,DP02_0040E,FIPS_State
1,Arkansas,38542,38542,0.56,05
2,Washington,96936,96936,0.55,53
3,Kansas,38807,38807,0.59,20
4,Oklahoma,54037,54037,0.59,40
5,Wisconsin,68361,68361,0.53,55


# 6. Plot

### Q1

In [21]:
df_Q1 = df.sort_values(by="DP02_0040E", ascending=False)

print(df_Q1.shape)
df_Q1.head()

(50, 5)


,State_Name,DP02_0037E,DP02_0037PE,DP02_0040E,FIPS_State
28,North Dakota,11331,11331,0.66,38
17,South Dakota,12569,12569,0.66,46
34,Wyoming,8279,8279,0.65,56
22,Utah,50568,50568,0.64,49
45,Nebraska,27343,27343,0.63,31


In [22]:
fig = px.bar(df_Q1,              
             x='State_Name', 
             y='DP02_0040E',
             text='DP02_0040E',
             orientation='v',   
             template='plotly_dark',
             title='Q1: All states sorted by DP02_0040E')

fig.update_traces(textposition='auto', 
                  texttemplate='%{text:,.0%}'   # This adds commas and formats to zero decimal places
                 )

fig.show()

### Q2

In [23]:
df_Q2 = df_Q1.iloc[ : 5]
df_Q2.sort_values(by="DP02_0040E", ascending=True, inplace=True)
df_Q2['DP02_0040E'] = df_Q2['DP02_0040E'] * 100

C:\Users\W10\AppData\Local\Temp\ipykernel_9468\2497920499.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\W10\AppData\Local\Temp\ipykernel_9468\2497920499.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
fig = px.bar(df_Q2,              
             x='DP02_0040E', 
             y='State_Name',
             text='DP02_0040E',
             orientation='h',   
             template='plotly_white',
             title='Top 5 States for Number of women 15 to 50 years who had a birth in the past 12 months')

fig.update_traces(textposition='auto', 
                  texttemplate='%{text:,.0f}'   # This adds commas and formats to zero decimal places
                 )

fig.show()

### Q3

In [25]:
df_Q3 = df.sort_values(by="DP02_0040E", ascending=True)
df_Q3 = df_Q3.iloc[ : 5]

In [26]:
fig = px.bar(df_Q3,              
             x='DP02_0040E', 
             y='State_Name',
             text='DP02_0040E',
             orientation='h',   
             template='plotly_dark',
             title='Q3: Bottom 5 states sorted by DP02_0040E')

fig.update_traces(textposition='auto', 
                  texttemplate='%{text:,.0%}'   # This adds commas and formats to zero decimal places
                 )

fig.show()